In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("pipeline").getOrCreate()

23/12/28 11:19:48 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/28 11:19:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 11:19:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/28 11:19:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [3]:
from pyspark.ml.feature import VectorAssembler

vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

In [4]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="dependente")

In [5]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp=MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,4,3], featuresCol="independente", labelCol="dependente")

In [6]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vector, indexer, mlp])
modelo = pipeline.fit(irisTreino)

In [7]:
previsao = modelo.transform(irisTeste)

In [8]:
previsao.show()

+-----------+----------+-----------+----------+---------------+-----------------+----------+--------------------+--------------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|          class|     independente|dependente|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+---------------+-----------------+----------+--------------------+--------------------+----------+
|        4.3|       3.0|        1.1|       0.1|    Iris-setosa|[4.3,3.0,1.1,0.1]|       2.0|[-309.86094351985...|[8.41984814052975...|       2.0|
|        4.4|       3.2|        1.3|       0.2|    Iris-setosa|[4.4,3.2,1.3,0.2]|       2.0|[-309.86094351988...|[8.41984814003677...|       2.0|
|        4.5|       2.3|        1.3|       0.3|    Iris-setosa|[4.5,2.3,1.3,0.3]|       2.0|[-309.86094351986...|[8.41984814040818...|       2.0|
|        4.6|       3.2|        1.4|       0.2|    Iris-setosa|[4.6,3.2,1.4,0.2]|       2.0|[-309.86094351989...|[8.41984813

In [10]:
previsao.select("independente", "dependente", "rawPrediction", "probability", "prediction").show()

+-----------------+----------+--------------------+--------------------+----------+
|     independente|dependente|       rawPrediction|         probability|prediction|
+-----------------+----------+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|       2.0|[-309.86094351985...|[8.41984814052975...|       2.0|
|[4.4,3.2,1.3,0.2]|       2.0|[-309.86094351988...|[8.41984814003677...|       2.0|
|[4.5,2.3,1.3,0.3]|       2.0|[-309.86094351986...|[8.41984814040818...|       2.0|
|[4.6,3.2,1.4,0.2]|       2.0|[-309.86094351989...|[8.41984813990372...|       2.0|
|[4.8,3.0,1.4,0.1]|       2.0|[-309.86094351989...|[8.41984813990372...|       2.0|
|[4.8,3.0,1.4,0.3]|       2.0|[-309.86094351989...|[8.41984813986352...|       2.0|
|[4.8,3.4,1.6,0.2]|       2.0|[-309.86094351990...|[8.41984813978885...|       2.0|
|[4.8,3.4,1.9,0.2]|       2.0|[-309.86094351990...|[8.41984813976014...|       2.0|
|[4.9,3.0,1.4,0.2]|       2.0|[-309.86094351989...|[8.41984813985586...|    

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)

print(f"Acuracia: {acuracia}")

Acuracia: 0.9523809523809523
